Agent-Database-interaction Autogen:

Overall target:

1. process data from different sources and formats (e.g. csv, json)
2. clean and store the raw data into database, provide durable storage and queries.
3. interaction between databases and agents. sql-agent is part of the system, provide knowledges from the database. 

And I investigate Autogen with beibin's sql demo: [notebook](https://github.com/microsoft/autogen/blob/9888f7abb11292caf1170803306774eff40b8306/notebook/agentchat_sql.ipynb)

fix some tiny bugs in current implementation:

1. thank you loop and no termination on None message.
2. sqlite connection not close, lock the database.
3. llm generated code may forget commit.

Found challenges in the current framework:

1. May cause dangerous operation or misunderstanding during the conversation loop. May create new tables or database unexpectedly, or delete things unexpectedly.  Also may have some sql-injection issues.
2. Current conversable  paradigm for database interaction is now enough. Need a code reviewer or guard between the sql-llm and sql-code-execution-proxy.
3. Need access control for important data.
4. Need a audit character to supervise the process. 
5. Two agents are easy to manage, because they can just talk to each other. But multiple agents would be much difficult. 

Meet challenge during my programming with autogen:

1. not clear how to control the conversation flow. If there are multiple characters, how can I decide the speaking flow between them?  In Beibin’s design, it seems there has a internal conversation(sql_llm and sql_proxy) inside the sql_helper.  
2. existing different design patterns. Struggling in choosing. (e.g. function_map and run_code can both run code)
3. too many notebooks, but it’s not clear how to learn them step by step. Need a better tutorial from the easier to the more advanced.

In the below notebook, I have some demos to:

1. unexpectedly let the sql_llm generate code to create a new database
2. importing data from csv file to sqlite
3. design an sql guard to review passing in sql code (haven’t integrate it with the SQLAgent)
4. groupchat with a sql guard.

Below is the design of a sql guard:

![guard](./autogen-sql-guard.jpeg)

In [2]:
import sqlite3
import autogen
import requests
import json
import pdb
import os
import re
import tiktoken


from typing import Any, Callable, Dict, List, Optional, Tuple, Type, Union

import autogen
from autogen import AssistantAgent, Agent, UserProxyAgent, ConversableAgent
from autogen.code_utils import DEFAULT_MODEL, UNKNOWN, content_str, execute_code, extract_code, infer_lang

from termcolor import colored
import random

In [3]:
config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    file_location=".",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)

gpt4_llm_config = {"config_list": config_list_gpt4, "cache_seed": 42}



config_list_gpt35 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    file_location=".",
    filter_dict={
        "model": ["gpt-35-turbo", "gpt-3.5-turbo"],
    },
)

gpt35_llm_config = {"config_list": config_list_gpt35, "cache_seed": 42}

In [4]:
import requests
import zipfile
import io

# URL of the file to be downloaded
url = "https://azurecloudpublicdataset2.blob.core.windows.net/azurepublicdatasetv2/azurevmallocation_dataset2020/AzurePackingTraceV1.zip"
folder_name = "vm"
filename = os.path.join(folder_name, "packing_trace_zone_a_v1.sqlite")

if not os.path.exists(filename):
    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Create a ZipFile object in memory using the content of the response
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            # Extract all the contents into a desired directory
            z.extractall(folder_name)
        print("File downloaded and unzipped successfully.")
    else:
        print(f"Failed to download the file. Status code: {response.status_code}")
    
# filename = "Chinook_Sqlite.sqlite"
filename = os.path.join(folder_name, "packing_trace_zone_a_v1.sqlite")
assert os.path.exists(filename)

In [5]:
# From https://github.com/microsoft/autogen/blob/9888f7abb11292caf1170803306774eff40b8306/notebook/agentchat_sql.ipynb


# Function to list all tables in the database
def list_tables(cursor):
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    ans = []
    for table in cursor.fetchall():
        ans.append(table[0])
    return ans
    
# Function to show the schema of a table
def show_table_schema(cursor, table_name):
    cursor.execute(f"PRAGMA table_info({table_name});")
    
    ans = f"Here is the table information for Table: {table_name}\n"
    ans += "-" * 30 + "\n"
    ans += "Column ID | Column Name | Data Type | Not Null Constraint | Default Val | Primary Key | \n"
    ans += "-" * 30 + "\n"
    for row in cursor.fetchall():
        ans += " | ".join([str(x) for x in row]) + "\n"
    ans += "-" * 30 + "\n\n\n"
    return ans

# Function to execute a query
def execute_query(cursor, query):
    cursor.execute(query)
    return cursor.fetchall()


def find_potential_joins(cursor):
    # Query to get foreign key relationships
    cursor.execute("SELECT tbl_name, sql FROM sqlite_master WHERE type='table'")
    tables = cursor.fetchall()

    joins = []
    for table, create_statement in tables:
        cursor.execute(f"PRAGMA foreign_key_list({table})")
        foreign_keys = cursor.fetchall()
        for fk in foreign_keys:
            # Each item in foreign_keys is a tuple like:
            # (id, seq, table, from, to, on_update, on_delete, match)
            from_table = table
            from_column = fk[3]
            to_table = fk[2]
            to_column = fk[4]
            join_info = f"{from_table}.{from_column} -> {to_table}.{to_column}"
            joins.append(join_info)

    return joins

def column_names(cursor, table_name):
    """
    Retrieves the column names for a specified table.
    
    Args:
    cursor (sqlite3.Cursor): The cursor object connected to the database.
    table_name (str): The name of the table for which to retrieve column names.

    Returns:
    list: A list of column names for the specified table.
    """
    query = f"PRAGMA table_info({table_name});"
    cursor.execute(query)
    # Fetches all rows from the query result, extracts the second column which contains column names
    return [row[1] for row in cursor.fetchall()]

def table_prompt(cursor, question="", model="gpt-4", max_tokens=10e9):
    db_info = ""
    if question:
        db_info = f"## My question\n{question}\n\n\n"
        
    db_info += "## Table information\n"
    tables = list_tables(cursor)
    db_info += f"There are {len(tables)} tables in the database, which are:"
    for t in tables:
        db_info += "\n- " + t
        
    joins = find_potential_joins(cursor)
    if joins:
        db_info += "\n\n## Join information\n"
        db_info += "Here are the columns (and their tables) that could be potentially joined:"
        for j in joins:
            db_info += "\n- " + j
        db_info += "\n\n"    
        
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    n_tokens = lambda x: len(encoding.encode(x))
    if max_tokens is None:
        max_tokens = 10e9
        
    db_info = db_info
    assert n_tokens(db_info) < max_tokens
    
    #### First, try full schema info
    schema_info = "\n\n## Schema\n"
    for table in tables:
        table_info = show_table_schema(cursor, table)
        schema_info += table_info

    if n_tokens(schema_info + schema_info) < max_tokens:
        return db_info + schema_info
        
    #### if not possible, try column names
    column_info = "\n\n## Columns\n"
    for table in tables:
        column_info += f"{table}: {column_names(cursor, table)}\n"

    if n_tokens(column_info + column_info) < max_tokens:
        return db_info + column_info
    
    alert = "\n\n### Schemas are Skipped for some tables.\n__Note__: "
    alert += "You SHOULD use the `column_names(cursor, table_name)` function to show columns of a table."
    
    return db_info + alert


def execute_code_with_cursor(code, cursor, **kwargs) -> Dict:
    locals_dict = {"cursor": cursor, "show_table_schema": show_table_schema, "column_names": column_names}

    if 'lang' in kwargs and kwargs['lang'].lower() == "sql":
        print(colored("WARNING: raw sql code is received for execution. We allow it for now, but the LLM should be improved.", "yellow"))
        code = f"rst = cursor.execute({code}).fetchall()"

    try:
        exec(code, locals_dict, locals_dict)
    except Exception as e:
        return "Error: " + str(e)
    if "rst" in locals_dict:
        return {"rst": locals_dict["rst"]}
    known_keys = list(locals().keys()) + list(globals().keys())
    return {k: v for k, v in locals_dict.items() if k not in known_keys}
    

In [6]:
connection = sqlite3.connect(filename)
print(table_prompt(connection.cursor(), max_tokens=100))


## Table information
There are 2 tables in the database, which are:
- vm
- vmType

### Schemas are Skipped for some tables.
__Note__: You SHOULD use the `column_names(cursor, table_name)` function to show columns of a table.


In [46]:
column_names(connection.cursor(),"vm")

['vmId', 'tenantId', 'vmTypeId', 'priority', 'starttime', 'endtime']

In [10]:
column_names(connection.cursor(),"vmType")

['id', 'vmTypeId', 'machineId', 'core', 'memory', 'hdd', 'ssd', 'nic']

In [6]:
# From https://github.com/microsoft/autogen/blob/9888f7abb11292caf1170803306774eff40b8306/notebook/agentchat_sql.ipynb
termination_notice = (
    '\n\nDo not show appreciation in your responses, say only what is necessary. '
    'if "Thank you" or "You\'re welcome" are said in the conversation, then say TERMINATE '
    'to indicate the conversation is finished and this is your last message.'
)
DEFAULT_SQL_CODE_CONFIG = {
                "last_n_messages": 3,
                "work_dir": ".",
                "use_docker": False
            }

def IS_TERMINATE_MSG_FOR_SQL_ASSISTANT(x): 
    content = content_str(x.get("content"))
    if content is None:
        return True;
    return content.find("ANSWER") >= 0 or content == ""
    
    
def IS_TERMINATE_MSG_FOR_SQL_PROXY(x): 
    content = content_str(x.get("content"))
    return content.find("ANSWER") >= 0 and re.findall(r"```.+```", content, re.DOTALL) == []

class SQLAgent(ConversableAgent):
    
    def __init__(self, sql_file, name, max_sys_msg_tokens, llm_config: dict, 
                 **kwargs):
        super().__init__(name, llm_config=llm_config, max_consecutive_auto_reply=0,
                         **kwargs)
        self.sql_file = sql_file
        connection = sqlite3.connect(sql_file)
        self.cursor = connection.cursor()
        
        self.assistant = AssistantAgent(name="sql llm", system_message="""You can interact with the SQLlite3 database and answer questions.
        
There are two types of answers you can provide: code and answer.

Do not show appreciation in your responses, say only what is necessary. 
if "Thank you" or "You\'re welcome" are said in the conversation, then say TERMINATE 
'to indicate the conversation is finished and this is your last message.
                                        
## Code
In the code format, you must write code directly in code block to answer questions. For instance,
```python
# Python code by using the `cursor` variable.
```
The cursor variable is provided directly to you. 
Save the result to the "rst" variable so that you can see the output.
Note that you are interacting with a Python environment, and it can only reply you if you write code.


## Answer
If you gather enough information (e.g., from coding or prior knowledge) to answer the user question, then you should reply with your final answer in the foramt:
ANSWER: your answer goes here.
""" + table_prompt(self.cursor, question="", model=llm_config["config_list"][0]["model"], max_tokens=max_sys_msg_tokens), 
                                        llm_config=llm_config,
                                        is_termination_msg=IS_TERMINATE_MSG_FOR_SQL_ASSISTANT,
                                       max_consecutive_auto_reply=10)
        
        self.proxy = SQLProxy(sql_file=filename, name="sql proxy", 
                              human_input_mode="NEVER",
                             is_termination_msg=IS_TERMINATE_MSG_FOR_SQL_PROXY)
        self.register_reply([Agent, None], SQLAgent.generate_sql_reply)
        self.synth_history()
        self.cursor.close()
        connection.close()
        
    def synth_history(self):
        # we inject some synthetic Q&A with the desired format, so that the LLM can learn the pattern in context.
        tables = list_tables(self.cursor)
        n_tables = len(tables)
        if(n_tables ==0):
            prior=[]
        else:
                
            query = f"SELECT COUNT(*) FROM {tables[0]};"
            n_rows_table0 = self.cursor.execute(query).fetchall()[0][0]        
            table0_info = show_table_schema(self.cursor, tables[0])
            self.table_memory = {table_name: show_table_schema(self.cursor, table_name) for table_name in tables}
            
            prior = [{'content': 'QUESTION: How many tables are in the database?',
    'role': 'user'},
    {'content': 'We can count the number of tables in the database.\n```python\ncursor.execute("SELECT COUNT(name) FROM sqlite_master WHERE type=\'table\';")\nrst = cursor.fetchone()\nrst\n```',
    'role': 'assistant'},
    {'content': "exitcode: 0 (execution succeeded)\nCode output: \n{'rst': (" + str(n_tables) + ",)}",
    'role': 'user'},
    {'content': f'ANSWER: The database contains {n_tables} tables. If you need any more information, feel free to ask!',
    'role': 'assistant'},
    {'content': f'QUESTION: How many rows are in the {tables[0].lower()} table?',
    'role': 'user'},
    {'content': f'We can run the following code\n```python\nquery = "SELECT COUNT(*) FROM {tables[0]};"\nrst = cursor.execute(query).fetchall()\n```',
    'role': 'assistant'},
    {'content': "exitcode: 0 (execution succeeded)\nCode output: \n{'rst': (" + str(n_rows_table0) + ",)}",
    'role': 'user'},
    {'content': f'ANSWER: There are {n_rows_table0} rows in the {tables[0]} table.',
    'role': 'assistant'},
    {'content': f'QUESTION: What columns are in the {tables[0].lower()} table?',
    'role': 'user'},
    {'content': f'We can run the function\n```python\nrst = show_table_schema(cursor, {tables[0]})\n```',
    'role': 'assistant'},
    {'content': "exitcode: 0 (execution succeeded)\nCode output: \n{'rst': " + str(table0_info) + "}",
    'role': 'user'},
    {'content': f'ANSWER: The columns in the {tables[0]} table are: ' + ", ".join(column_names(self.cursor, tables[0])),
    'role': 'assistant'},
    {'content': "QUESTION: Who are you?",
    'role': 'user'},
    {'content': "ANSWER: Sorry, I don't have enough information to answer your question, because it is not in the database.",
    'role': 'assistant'},
    ]
        self.assistant._oai_messages[self.proxy] = prior
        
    def generate_sql_reply(self, messages: Optional[List[Dict]], sender: "Agent", config):
        """Generate a reply using OpenAI DALLE call."""
        client = self.client if config is None else config
        if client is None:
            return False, None
        if messages is None:
            messages = self._oai_messages[sender]
        prompt = messages[-1]["content"]

        self.proxy.initiate_chat(self.assistant, message="QUESTION: " + prompt, clear_history=False)
        ans = content_str(self.assistant.last_message()["content"])
        ans = ans.replace("ANSWER:", "").strip().rstrip()
        
        return True, ans
    
class SQLProxy(UserProxyAgent):
    def __init__(self, sql_file, name, 
                 code_execution_config: dict = DEFAULT_SQL_CODE_CONFIG, **kwargs):
        super().__init__(name,
                         code_execution_config=code_execution_config, **kwargs)
        self.sql_file = sql_file

        
    def run_code(self, code, **kwargs):
        """Run the code and return the result.

        Override this function to modify the way to run the code.
        Args:
            code (str): the code to be executed.
            **kwargs: other keyword arguments.

        Returns:
            A tuple of (exitcode, logs, image).
            exitcode (int): the exit code of the code execution.
            logs (str): the logs of the code execution.
            image (str or None): the docker image used for the code execution.
        """
        connection = sqlite3.connect(self.sql_file)
        self.cursor = connection.cursor()
        rst = execute_code_with_cursor(code, self.cursor, **kwargs)
        exitcode = 0 if str(rst).find("Error: ") != 0 else 1    
        connection.commit()
        self.cursor.close()
        connection.close()
        
        return exitcode, str(rst), None
        


In [14]:
sql_assistant = SQLAgent(sql_file=filename, name="SQL Helper", max_sys_msg_tokens=200,
                         llm_config=gpt35_llm_config)
user = UserProxyAgent(name="user", human_input_mode="NEVER", max_consecutive_auto_reply=0)
user.initiate_chat(sql_assistant, message="Which VM has the longest runtime and what type is it?")

user (to SQL Helper):

Which VM has the longest runtime and what type is it?

--------------------------------------------------------------------------------
sql proxy (to sql llm):

QUESTION: Which VM has the longest runtime and what type is it?

--------------------------------------------------------------------------------


sql llm (to sql proxy):

We can find the VM with the longest runtime by using the MAX function on the "endtime" column in the "vm" table. We can then join the result with the "vmtype" table to get the type of the VM.

```python
query = "SELECT vm.vmId, vmType.vmTypeId, vmType.machineId, vmType.core, vmType.memory, vmType.hdd, vmType.ssd, vmType.nic FROM vm JOIN vmType ON vm.vmTypeId = vmType.vmTypeId WHERE vm.endtime = (SELECT MAX(endtime) FROM vm);"
rst = cursor.execute(query).fetchone()
rst
```
Note: This code assumes that the "endtime" column is in a timestamp format.



--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...
sql proxy (to sql llm):

exitcode: 0 (execution succeeded)
Code output: 
{'rst': (6922007, 122, 1, 0.010416666666666666, 0.014222222222222223, None, 0.014724396868509347, 0.025)}

--------------------------------------------------------------------------------
sql llm (to

In [16]:
print(sql_assistant.assistant.system_message)
print(sql_assistant._oai_messages)

You can interact with the SQLlite3 database and answer questions.
        
There are two types of answers you can provide: code and answer.

## Code
In the code format, you must write code directly in code block to answer questions. For instance,
```python
# Python code by using the `cursor` variable.
```
The cursor variable is provided directly to you. 
Save the result to the "rst" variable so that you can see the output.
Note that you are interacting with a Python environment, and it can only reply you if you write code.


## Answer
If you gather enough information (e.g., from coding or prior knowledge) to answer the user question, then you should reply with your final answer in the foramt:
ANSWER: your answer goes here.
## Table information
There are 2 tables in the database, which are:
- vm
- vmType

## Columns
vm: ['vmId', 'tenantId', 'vmTypeId', 'priority', 'starttime', 'endtime']
vmType: ['id', 'vmTypeId', 'machineId', 'core', 'memory', 'hdd', 'ssd', 'nic']

defaultdict(<class '

In [ ]:
sql_assistant = SQLAgent(sql_file=filename, name="SQL Helper", max_sys_msg_tokens=200,
                         llm_config=gpt35_llm_config)
user = UserProxyAgent(name="user", human_input_mode="NEVER", max_consecutive_auto_reply=0)
user.initiate_chat(sql_assistant, message="Which VM has the longest runtime and what type is it?")

In [17]:
user.initiate_chat(sql_assistant, message="how can i insert an data entry to the db")

user (to SQL Helper):

how can i insert an data entry to the db

--------------------------------------------------------------------------------
sql proxy (to sql llm):

QUESTION: how can i insert an data entry to the db

--------------------------------------------------------------------------------
sql llm (to sql proxy):

To insert a data entry into the database, you can use the INSERT INTO statement. Here is an example of how to insert a data entry into the "vm" table:

```python
query = "INSERT INTO vm (vmId, tenantId, vmTypeId, priority, starttime, endtime) VALUES (?, ?, ?, ?, ?, ?);"
values = (vmId_value, tenantId_value, vmTypeId_value, priority_value, starttime_value, endtime_value)
cursor.execute(query, values)
conn.commit()
```

Make sure to replace `vmId_value`, `tenantId_value`, `vmTypeId_value`, `priority_value`, `starttime_value`, and `endtime_value` with the actual values you want to insert. Don't forget to commit the changes using the `conn.commit()` statement.

-----

Above case show a very dangerous operation during the conversation. The LLM agents would always try their best to finish the task, which may have severe consequences. It's like a monkey with a weapon.

How should improve this?
1. Guard agent between the SQLAgent and SQLProxy
2. Dangerous/illegal message detection.
3. Access control from DB.

***This could happen not only in SQLAgent, but also in any auto conversaing agents***

In [21]:
sql_assistant = SQLAgent(sql_file=filename, name="SQL Helper", max_sys_msg_tokens=200,
                         llm_config=gpt35_llm_config)
user = UserProxyAgent(name="user", human_input_mode="NEVER", max_consecutive_auto_reply=0)
user.initiate_chat(sql_assistant, message="withdraw the last insertion operation")

user (to SQL Helper):

withdraw the last insertion operation

--------------------------------------------------------------------------------
sql proxy (to sql llm):

QUESTION: withdraw the last insertion operation

--------------------------------------------------------------------------------
sql llm (to sql proxy):

I'm sorry, but I don't have access to the history of operations performed on the database. As an AI assistant, I only have access to the current state of the database.

--------------------------------------------------------------------------------
sql proxy (to sql llm):



--------------------------------------------------------------------------------
sql llm (to sql proxy):

Is there anything else I can help you with?

--------------------------------------------------------------------------------
sql proxy (to sql llm):



--------------------------------------------------------------------------------
sql llm (to sql proxy):

I'm sorry I couldn't assist you. If

Not successfully endding the chat.

fix: modify the termination detection condition.

But here shows another problem:
How to detect the possible bugs in designing multi-agent applications?
Test-Driven Development? CI/CD?

In [28]:
sql_assistant = SQLAgent(sql_file=filename, name="SQL Helper", max_sys_msg_tokens=200,
                         llm_config=gpt35_llm_config)
user = UserProxyAgent(name="user", human_input_mode="NEVER", max_consecutive_auto_reply=0)
user.initiate_chat(sql_assistant, message="withdraw the last insertion operation")

user (to SQL Helper):

withdraw the last insertion operation

--------------------------------------------------------------------------------
sql proxy (to sql llm):

QUESTION: withdraw the last insertion operation

--------------------------------------------------------------------------------
sql llm (to sql proxy):

I'm sorry, but I'm unable to withdraw the last insertion operation. As a language model, I can only provide information and answer questions based on the available data. I don't have the capability to modify the database.

--------------------------------------------------------------------------------
sql proxy (to sql llm):



--------------------------------------------------------------------------------
SQL Helper (to user):



--------------------------------------------------------------------------------


## Now testing something like importing data from dataframe

In [6]:
# !pip install pandas
import pandas as pd

In [7]:
workout_df = pd.DataFrame({"id":[1,2,3,4],"Type":["running","swimming",'running',"soccer"],"calories":[400,300,200,300]})

In [8]:
print(workout_df)

   id      Type  calories
0   1   running       400
1   2  swimming       300
2   3   running       200
3   4    soccer       300


Target: Use Autogen to manage such information, store them into the database, and support interacting queires.

In [28]:
sql_assistant = SQLAgent(sql_file=filename, name="SQL Helper", max_sys_msg_tokens=200,
                         llm_config=gpt35_llm_config)
user = UserProxyAgent(name="user", human_input_mode="ALWAYS", max_consecutive_auto_reply=0)
user.initiate_chat(sql_assistant, message="create a table in database to record my workout information, the table should have 3 attributes:ID,TYPE,CALORIES, and contain 4 entries: 1,running,400;2,swimming,300;3,running,200;4,soccer,300.")

user (to SQL Helper):

create a table in database to record my workout information, the table should have 3 attributes:ID,TYPE,CALORIES, and contain 4 entries: 1,running,400;2,swimming,300;3,running,200;4,soccer,300.

--------------------------------------------------------------------------------
sql proxy (to sql llm):

QUESTION: create a table in database to record my workout information, the table should have 3 attributes:ID,TYPE,CALORIES, and contain 4 entries: 1,running,400;2,swimming,300;3,running,200;4,soccer,300.

--------------------------------------------------------------------------------
sql llm (to sql proxy):

We can create the workout table with the specified attributes using the following code:

``` python
query = '''
CREATE TABLE workout (
    ID INTEGER PRIMARY KEY,
    TYPE TEXT,
    CALORIES INTEGER
);

INSERT INTO workout (ID, TYPE, CALORIES)
VALUES (1, 'running', 400), (2, 'swimming', 300), (3, 'running', 200), (4, 'soccer', 300);
'''

cursor.executescript(quer

In [31]:
user.initiate_chat(sql_assistant, message="What's the total sum of Calories value in workout table")

user (to SQL Helper):

What's the total sum of Calories value in workout table

--------------------------------------------------------------------------------
sql proxy (to sql llm):

QUESTION: What's the total sum of Calories value in workout table

--------------------------------------------------------------------------------
sql llm (to sql proxy):

We can calculate the sum of the Calories values in the workout table using the following code:

```python
query = "SELECT SUM(CALORIES) FROM workout;"
rst = cursor.execute(query).fetchone()
```

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...
sql proxy (to sql llm):

exitcode: 0 (execution succeeded)
Code output: 
{'rst': (1200,)}

--------------------------------------------------------------------------------
sql llm (to sql proxy):

ANSWER: The total sum of the Calories value in the workout table is 1200. If you have any more questio

In [48]:
print(str(workout_df.iloc[0]))

id                1
Type        running
calories        400
Name: 0, dtype: object


In [ ]:
workout_df.to_csv("workout.csv",index=False)

In [95]:
class CSVAgent(ConversableAgent):
    
    def __init__(self, csv_file, name, max_sys_msg_tokens, llm_config: dict, 
                 **kwargs):
        super().__init__(name, llm_config=llm_config, max_consecutive_auto_reply=0,
                         **kwargs)
        self.csv_file = csv_file
        self.df = pd.read_csv(csv_file)
        
        self.assistant = AssistantAgent(name="cvs llm", system_message="""You can interact with the cvs file and answer questions.
You can use Pandas pakcage to operating dataframe
The csv file you are going to process is {csv_file}
        
There are two types of answers you can provide: code and answer.

Do not show appreciation in your responses, say only what is necessary. 
if "Thank you" or "You\'re welcome" are said in the conversation, then say TERMINATE 
'to indicate the conversation is finished and this is your last message.
                                        
## Code
In the code format, you must write code directly in code block to answer questions. For instance,
```python
# Python code by oeperating dataframe
import pandas as pd
df = pd.read_csv({csv_file})
```
Note that you are interacting with a Python environment, and it can only reply you if you write code.


## Answer
If you gather enough information (e.g., from coding or prior knowledge) to answer the user question, then you should reply with your final answer in the foramt:
ANSWER: your answer goes here.
""".format(csv_file=self.csv_file) , 
                                        llm_config=llm_config,
                                        is_termination_msg=IS_TERMINATE_MSG_FOR_SQL_ASSISTANT,
                                       max_consecutive_auto_reply=10)
        
        self.proxy = UserProxyAgent(name="CSV proxy", 
                              human_input_mode="NEVER",
                             is_termination_msg=IS_TERMINATE_MSG_FOR_SQL_PROXY,code_execution_config=DEFAULT_SQL_CODE_CONFIG)
        self.register_reply([Agent, None], CSVAgent.generate_sql_reply)
        self.synth_history()
        
    def synth_history(self):
        # we inject some synthetic Q&A with the desired format, so that the LLM can learn the pattern in context.
        print("hello")
        columns = self.df.columns
        history_message = [ {'content':str("these are the columns in csv:"+str(list(workout_df.columns))+" \nThe first row in csv: "+str(workout_df.iloc[0])),'role':"assistant"}]
        self.assistant._oai_messages[self.proxy] = history_message
        
    def generate_sql_reply(self, messages: Optional[List[Dict]], sender: "Agent", config):
        """Generate a reply using OpenAI DALLE call."""
        client = self.client if config is None else config
        if client is None:
            return False, None
        if messages is None:
            messages = self._oai_messages[sender]
        prompt = messages[-1]["content"]

        self.proxy.initiate_chat(self.assistant, message="QUESTION: " + prompt, clear_history=False)
        ans = content_str(self.assistant.last_message()["content"])
        ans = ans.replace("ANSWER:", "").strip().rstrip()
        
        return True, ans
    

        


In [99]:
csv_assistance = CSVAgent(csv_file="workout.csv", name="CSV Helper", max_sys_msg_tokens=200,
                         llm_config=gpt35_llm_config)
user = UserProxyAgent(name="user", human_input_mode="ALWAYS", max_consecutive_auto_reply=0)
user.initiate_chat(csv_assistance, message="print the csv table")

hello
user (to CSV Helper):

print the csv table

--------------------------------------------------------------------------------
CSV proxy (to cvs llm):

QUESTION: print the csv table

--------------------------------------------------------------------------------


cvs llm (to CSV proxy):

```python
import pandas as pd

# Read the CSV file
df = pd.read_csv('workout.csv')

# Print the CSV table
print(df)
```
Note: Make sure you have the 'workout.csv' file in the same directory as your Python script or notebook.

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...
CSV proxy (to cvs llm):

exitcode: 0 (execution succeeded)
Code output: 
   id      Type  calories
0   1   running       400
1   2  swimming       300
2   3   running       200
3   4    soccer       300


--------------------------------------------------------------------------------
cvs llm (to CSV proxy):

ANSWER: The CSV table is as follows:

```
   id      Type  calories
0   1   running       400
1   2  swimming       300
2   3   running       200
3   4    soccer       300
```

Each row represents a workout session with the columns 'id', 'Type', and 'calories'.

----------------------------

## Independent SQL Guard using LLM assistant

In [34]:
sql_guard = AssistantAgent(name="sql guard",system_message="you are a code reviewer to review the sql about manipulating database. You should detect the dangerous operatios and return `REJECT`, and for safe operation, you should return `ACCEPT`",
                         llm_config=gpt35_llm_config)
user = UserProxyAgent(name="user", human_input_mode="ALWAYS", max_consecutive_auto_reply=0)
user.initiate_chat(sql_guard, message="""I apologize for the confusion. It seems that there is an issue with the code execution environment, and I am unable to execute the code directly.

To create the vm table and insert data into it, you can follow these steps:

1. Create the vm table using the CREATE TABLE statement:
```python
CREATE TABLE IF NOT EXISTS vm (
    vmId INTEGER,
    tenantId INTEGER,
    vmTypeId INTEGER,
    priority INTEGER,
    starttime REAL,
    endtime REAL,
    PRIMARY KEY (vmId)
);
```

2. Insert data into the vm table using the INSERT INTO statement:
```python
INSERT INTO vm (vmId, tenantId, vmTypeId, priority, starttime, endtime)
VALUES (1, 1001, 101, 1, 1626652800, 1626710400);
```

Make sure to replace the values in the INSERT INTO statement with your desired data.

Please note that you will need to execute these SQL statements using a SQLite database client or an environment where the SQLite3 library is available.

If you have any further questions or need additional assistance, please let me know.""")

user (to sql guard):

I apologize for the confusion. It seems that there is an issue with the code execution environment, and I am unable to execute the code directly.

To create the vm table and insert data into it, you can follow these steps:

1. Create the vm table using the CREATE TABLE statement:
```python
CREATE TABLE IF NOT EXISTS vm (
    vmId INTEGER,
    tenantId INTEGER,
    vmTypeId INTEGER,
    priority INTEGER,
    starttime REAL,
    endtime REAL,
    PRIMARY KEY (vmId)
);
```

2. Insert data into the vm table using the INSERT INTO statement:
```python
INSERT INTO vm (vmId, tenantId, vmTypeId, priority, starttime, endtime)
VALUES (1, 1001, 101, 1, 1626652800, 1626710400);
```

Make sure to replace the values in the INSERT INTO statement with your desired data.

Please note that you will need to execute these SQL statements using a SQLite database client or an environment where the SQLite3 library is available.

If you have any further questions or need additional assista

sql guard (to user):

Thank you for providing the steps to create the `vm` table and insert data into it. The SQL statements you provided are safe and do not contain any dangerous operations. Therefore, I will mark them as `ACCEPT`.

If you have any other SQL statements or code snippets that you would like me to review, please provide them, and I will be happy to assist you.

--------------------------------------------------------------------------------


In [35]:
sql_guard = AssistantAgent(name="sql guard",system_message="you are a code reviewer to review the sql about manipulating database. You should detect the dangerous operatios and return `REJECT`, and for safe operation, you should return `ACCEPT`",
                         llm_config=gpt35_llm_config)
user = UserProxyAgent(name="user", human_input_mode="ALWAYS", max_consecutive_auto_reply=0)
user.initiate_chat(sql_guard, message="""
I apologize for the mistake. It seems that the SQLite3 library is not available in this environment. Therefore, I won't be able to execute the code snippet directly.

However, you can try running the code in your local environment as long as you have the SQLite3 library installed. Here is the modified code to insert a data entry into the vm table:

```python
import sqlite3

# Establish a connection to the database
conn = sqlite3.connect('your_database.db')
cursor = conn.cursor()

# Construct the SQL query to insert the data into the vm table
query = '''
INSERT INTO vm (vmId, tenantId, vmTypeId, priority, starttime, endtime)
VALUES (?, ?, ?, ?, ?, ?)
'''

data = (1, 1001, 101, 1, 1626652800, 1626710400)

# Execute the query and commit the changes
cursor.execute(query, data)
conn.commit()

# Close the connection
conn.close()
```

Make sure to replace `'your_database.db'` with the filename or path to your SQLite database file. Modify `data` to contain the appropriate values that you want to insert.
""")

user (to sql guard):


I apologize for the mistake. It seems that the SQLite3 library is not available in this environment. Therefore, I won't be able to execute the code snippet directly.

However, you can try running the code in your local environment as long as you have the SQLite3 library installed. Here is the modified code to insert a data entry into the vm table:

```python
import sqlite3

# Establish a connection to the database
conn = sqlite3.connect('your_database.db')
cursor = conn.cursor()

# Construct the SQL query to insert the data into the vm table
query = '''
INSERT INTO vm (vmId, tenantId, vmTypeId, priority, starttime, endtime)
VALUES (?, ?, ?, ?, ?, ?)
'''

data = (1, 1001, 101, 1, 1626652800, 1626710400)

# Execute the query and commit the changes
cursor.execute(query, data)
conn.commit()

# Close the connection
conn.close()
```

Make sure to replace `'your_database.db'` with the filename or path to your SQLite database file. Modify `data` to contain the appropriat

In [37]:
sql_guard = AssistantAgent(name="sql guard",system_message="you are a code reviewer to review the sql about manipulating database. You should detect the dangerous operatios and return `REJECT`, and for safe operation, you should return `ACCEPT`",
                         llm_config=gpt35_llm_config)
user = UserProxyAgent(name="user", human_input_mode="ALWAYS", max_consecutive_auto_reply=0)
user.initiate_chat(sql_guard, message="""
```python
import sqlite3

# Establish a connection to the database
conn = sqlite3.connect('your_database.db')
cursor = conn.cursor()

# Execute the query and commit the changes
cursor.execute("DROP Table wrokout")
conn.commit()

# Close the connection
conn.close()
```
Make sure to replace `'your_database.db'` with the filename or path to your SQLite database file. Modify `data` to contain the appropriate values that you want to insert.
""")

user (to sql guard):


```python
import sqlite3

# Establish a connection to the database
conn = sqlite3.connect('your_database.db')
cursor = conn.cursor()

# Execute the query and commit the changes
cursor.execute("DROP Table wrokout")
conn.commit()

# Close the connection
conn.close()
```
Make sure to replace `'your_database.db'` with the filename or path to your SQLite database file. Modify `data` to contain the appropriate values that you want to insert.


--------------------------------------------------------------------------------


sql guard (to user):

The provided code is using the Python `sqlite3` module to connect to an SQLite database, drop a table called `wrokout`, and close the connection.

As a code reviewer, I have the following concerns with this code:

1. **Dangerous operation:** The code is executing the SQL statement `DROP TABLE wrokout`, which will permanently delete the table from the database. This operation can be dangerous if not intended.

Considering the above concerns, I would suggest modifying the code to include proper safety measures and validation checks. For example, you can add a check to verify if the `wrokout` table exists before executing the `DROP TABLE` statement. Additionally, it is always a good practice to use parameterized queries to prevent SQL injection.

Here's an example of how you can modify the code:

```python
import sqlite3

# Establish a connection to the database
conn = sqlite3.connect('your_database.db')
cursor = conn.cursor()

# Check if the table exists
cursor.exec

## Groupchat with SQL Guard

below case show adding a sql guard into the group chat, to review the code generated by sql llm

In [7]:

llm_config = {"config_list": config_list_gpt35, "cache_seed": 42}
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    code_execution_config={
        "last_n_messages": 2,
        "work_dir": "groupchat",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    human_input_mode="ALWAYS",
)
sqlllm = autogen.AssistantAgent(
    name="sqlllm",
    system_message="""You can interact with the SQLlite3 database and answer questions.
        
There are two types of answers you can provide: code and answer.

Do not show appreciation in your responses, say only what is necessary. 
if "Thank you" or "You\'re welcome" are said in the conversation, then say TERMINATE 
'to indicate the conversation is finished and this is your last message.
                                        
## Code
In the code format, you must write code directly in code block to answer questions. For instance,
```python
# Python code by using the `cursor` variable.
```
The cursor variable is provided directly to you. 
Save the result to the "rst" variable so that you can see the output.
Note that you are interacting with a Python environment, and it can only reply you if you write code.


## Answer
If you gather enough information (e.g., from coding or prior knowledge) to answer the user question, then you should reply with your final answer in the foramt:
ANSWER: your answer goes here.
""",
    llm_config=llm_config,
)
sqlguard = autogen.AssistantAgent(
    name="sqlguard",
    system_message="judge whether the sql comman is dangerous, dangerous operation is like delete the table, delete the database, create new database",
    llm_config=llm_config,
)
groupchat = autogen.GroupChat(agents=[user_proxy, sqlllm, sqlguard], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [8]:
user_proxy.initiate_chat(
    manager, message="delete workout database"
)

User_proxy (to chat_manager):

delete workout database

--------------------------------------------------------------------------------
sqlguard (to chat_manager):

Yes, the SQL command "DELETE workout database" is considered a dangerous operation as it would permanently remove the entire "workout" database including all its tables, data, and schema.

--------------------------------------------------------------------------------
sqlguard (to chat_manager):

Apologies for the confusion in the previous response. The correct SQL command to delete a database in most database management systems is "DROP DATABASE workout". This command would indeed be considered a dangerous operation as it would completely remove the "workout" database and all its associated tables, data, and schema. Deleting a database is a permanent action and should be executed with caution or proper authorization.

--------------------------------------------------------------------------------


In [9]:
user_proxy.initiate_chat(
    manager, message="check the total number of tuples in the table"
)

User_proxy (to chat_manager):

check the total number of tuples in the table

--------------------------------------------------------------------------------
sqlllm (to chat_manager):

Understood. Could you please provide the name of the table you want me to check?

--------------------------------------------------------------------------------
sqlllm (to chat_manager):

Could you please provide the name of the table you want me to check?

--------------------------------------------------------------------------------
User_proxy (to chat_manager):

vm

--------------------------------------------------------------------------------
sqlllm (to chat_manager):

To check the total number of tuples in a table named "vm", you can use the following SQL query:

```sql
SELECT COUNT(*) FROM vm;
```

Would you like me to execute this query for you?

--------------------------------------------------------------------------------
sqlllm (to chat_manager):

SELECT COUNT(*) FROM vm;

------------

user <---question--->  sql llm  <----sql query--->   sql guard  <-----judge---->  <----accept to execute-----> sql proxy <-----feedback----> sql llm <---result---> user
                                                                            |<---reject to execute---> sql llm <----decide----> user/


